In [ ]:
import pandas as pd

base = "https://raw.githubusercontent.com/jourdee-lab/FYP_Data_Pipeline/64e81707fa5cec98445e6b48c671f1ad71a3f153/81sas04"
files = [f"{base}/81sas04ews_{i}.csv" for i in range(5)]

dfs = [pd.read_csv(url, dtype={"zoneid": "string"}) for url in files]
sas04_1981 = pd.concat(dfs, ignore_index=True)

# Clean column names and zoneid
sas04_1981.columns = sas04_1981.columns.str.strip()
sas04_1981["zoneid"] = sas04_1981["zoneid"].str.strip()
value_cols = [c for c in sas04_1981.columns if c != "zoneid"]
sas04_1981[value_cols] = sas04_1981[value_cols].apply(pd.to_numeric, errors="coerce")

# Specify Manchester district
MANCHESTER_DISTRICT = "03BN"
manchester_zones = sas04_1981.loc[
    sas04_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT),
    "zoneid"
].unique()

print(f"Found {len(manchester_zones)} Manchester zones")
print(f"Sample zones: {sorted(manchester_zones)[:5]}")

# Aggregate all Manchester zones
manchester_agg = pd.DataFrame(
    [sas04_1981.loc[sas04_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT), value_cols].sum(numeric_only=True)],
    columns=value_cols
)

manchester_agg.insert(0, "zoneid", MANCHESTER_DISTRICT)

print("\n=== Manchester City (1981) - SAS04 Country of Birth ===")
print(manchester_agg)

# Extracting far east births only   
far_east_cols = ["81sas040359", "81sas040360", "81sas040361"]
far_east_agg = manchester_agg[["zoneid"] + far_east_cols].copy()
far_east_agg.columns = ["zoneid", "far_east_total", "far_east_male", "far_east_female"]

print("\n=== Far East births in Manchester (1981) ===")
print(far_east_agg)
manchester_agg.to_csv("manchester_sas04_1981.csv", index=False)
print("\nFull aggregate saved to: manchester_sas04_1981.csv")


Found 1053 Manchester zones
Sample zones: ['03BN', '03BNAA', '03BNAA01', '03BNAA02', '03BNAA03']

=== Manchester City (1981) - SAS04 Country of Birth ===
  zoneid  81sas040320  81sas040321  81sas040322  81sas040323  81sas040324  \
0   03BN      1312954       638492       674462      1118981       535563   

   81sas040325  81sas040326  81sas040327  81sas040328  ...  81sas040370  \
0       583418        22704        11775        10929  ...         6873   

   81sas040371  81sas040372  81sas040373  81sas040374  81sas040375  \
0         6545         2756         3789         7433         3904   

   81sas040376  81sas040377  81sas040378  81sas040379  
0         3529        15051         9737         5314  

[1 rows x 61 columns]

=== Far East births in Manchester (1981) ===
  zoneid  far_east_total  far_east_male  far_east_female
0   03BN            7142           4511             2631

Full aggregate saved to: manchester_sas04_1981.csv
